In [1]:
!pip install langchain chainlit --quiet

In [6]:
!pip install langchain-community langchain-core --quiet
!pip install ctransformers --quiet

In [8]:
!pip install llama-cpp-python --quiet

In [6]:
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# Define the prompt template
template = """You are an intelligent AI assistant that identifies claims from a conversation. 
For each message in the conversation, return 1 if it is a claim and 0 if it is not.
Conversation: {Conversation}
Response format: {{1:0, 2:1, ...}}"""

prompt = PromptTemplate(template=template, input_variables=["Conversation"])

# Initialize the LLM
llm = CTransformers(model="models/llama-2-7b.ggmlv3.q4_0.bin", model_type='llama')

# Create the LLM chain
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

# Function to process a message
async def process_message(message: str):
    res = await llm_chain.acall({"Conversation": message})
    response_text = res['text']
    print("output is", response_text)
    # Extract and format the response into JSON
    try:
        response_json = json.loads(response_text)
    except json.JSONDecodeError:
        # Handle cases where the LLM response is not a valid JSON
        raise ValueError("LLM response is not a valid JSON: " + response_text)
    return response_json

# Example usage in the notebook
message = """{1: "Agent: Mike McNasby jil It looks like the volume of feedback coming from Discord has dropped to zero.",
            2:"The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?"} """
response = await process_message(message)
print("The response is ", response)




> Entering new LLMChain chain...
Prompt after formatting:
You are an intelligent AI assistant that identifies claims from a conversation. 
For each message in the conversation, return 1 if it is a claim and 0 if it is not.
Conversation: {1: "Agent: Mike McNasby jil It looks like the volume of feedback coming from Discord has dropped to zero.",
            2:"The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?"} 
Response format: {1:0, 2:1, ...}

> Finished chain.
output is   


ValueError: LLM response is not a valid JSON:   

In [11]:
! pip install mistralai --quiet

In [17]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv
import json

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

# Define the prompt template
template = """You are an intelligent AI assistant that identifies claims from a conversation. 
For each message in the conversation, return 1 if it is a claim and 0 if it is not.
Conversation: {Conversation}
Response format: {{1:0, 2:1,}}"""

# Function to create the message content based on the template
def create_message_content(conversation: str) -> str:
    return template.format(Conversation=conversation)

# Function to process a message
async def process_message(message: str):
    conversation = create_message_content(message)
    messages = [ChatMessage(role="user", content=conversation)]
    
    chat_response = client.chat(
        model=model,
        response_format={"type": "json_object"},
        messages=messages,
    )
    
    response_text = chat_response.choices[0].message.content
    print("output is", response_text)
    
    # Extract and format the response into JSON
    try:
        response_json = json.loads(response_text)
    except json.JSONDecodeError:
        # Handle cases where the Mistral response is not valid JSON
        raise ValueError("Mistral response is not valid JSON: " + response_text)
    
    return response_json

# Example usage in the notebook
message = """{1: "Agent: Mike McNasby jil It looks like the volume of feedback coming from Discord has dropped to zero.",
            2:"The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?"} """

response = await process_message(message)
print("The response is ", response)


output is {",": 0, ":": 0, "1": 1, "2": 1}
The response is  {',': 0, ':': 0, '1': 1, '2': 1}


In [19]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv
import json
import uuid

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

# Define the prompt template
template = """You are an intelligent AI assistant that identifies claims from a conversation. 
For each message in the conversation, return 1 if it is a claim and 0 if it is not.
Conversation: {Conversation}
Response format: {{1:0, 2:1, ...}}"""

# Function to create the message content based on the template
def create_message_content(conversation: str) -> str:
    return template.format(Conversation=conversation)

# Function to split the conversation into indices
def split_conversation(conversation: str) -> dict:
    messages = conversation.split("\n")
    indexed_messages = {i + 1: msg.strip() for i, msg in enumerate(messages)}
    return indexed_messages

# Function to process a message
async def process_message(message: str):
    conversation = create_message_content(message)
    messages = [ChatMessage(role="user", content=conversation)]
    
    chat_response = client.chat(
        model=model,
        response_format={"type": "json_object"},
        messages=messages,
    )
    
    response_text = chat_response.choices[0].message.content
    print("output is", response_text)
    
    # Extract and format the response into JSON
    try:
        response_json = json.loads(response_text)
    except json.JSONDecodeError:
        # Handle cases where the Mistral response is not valid JSON
        raise ValueError("Mistral response is not valid JSON: " + response_text)
    
    return response_json

# Function to identify claim indices
async def identify_claims(conversation: str):
    indexed_messages = split_conversation(conversation)
    indexed_messages_str = json.dumps(indexed_messages)
    print(indexed_messages_str)
    
    claims_result = await process_message(indexed_messages_str)
    
    claim_indices = [index for index, is_claim in claims_result.items() if is_claim == 1]
    
    return claim_indices

# Example usage in the notebook
conversation = """Agent: Mike McNasby: It looks like the volume of feedback coming from Discord has dropped to zero.
Agent: The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?
User: Yes, we intentionally reduced the monitoring on that channel.
User: We shifted our focus to other channels for bug reporting."""

record_id = str(uuid.uuid4())
claim_indices = await identify_claims(conversation)
output = {
    "record_id": record_id,
    "claim_indices": claim_indices
}

print("The response is ", output)


{"1": "Agent: Mike McNasby: It looks like the volume of feedback coming from Discord has dropped to zero.", "2": "Agent: The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?", "3": "User: Yes, we intentionally reduced the monitoring on that channel.", "4": "User: We shifted our focus to other channels for bug reporting."}
output is {"1": 0, "2": 1, "3": 1, "4": 1}
The response is  {'record_id': '7e11037d-bf1e-4055-af94-ec2fbf77e907', 'claim_indices': ['2', '3', '4']}


In [20]:
import os
import json
import uuid
import pandas as pd
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

# Define the prompt template
template = """You are an intelligent AI assistant that identifies claims from a conversation. 
For each message in the conversation, return 1 if it is a claim and 0 if it is not.
Conversation: {Conversation}
Response format: {{1:0, 2:1, ...}}"""

# Function to create the message content based on the template
def create_message_content(conversation: str) -> str:
    return template.format(Conversation=conversation)

# Function to split the conversation into indices
def split_conversation(conversation: str) -> dict:
    messages = conversation.split("\n")
    indexed_messages = {i + 1: msg.strip() for i, msg in enumerate(messages)}
    return indexed_messages

# Function to process a message
async def process_message(message: str):
    conversation = create_message_content(message)
    messages = [ChatMessage(role="user", content=conversation)]
    
    chat_response = client.chat(
        model=model,
        response_format={"type": "json_object"},
        messages=messages,
    )
    
    response_text = chat_response.choices[0].message.content
    print("output is", response_text)
    
    # Extract and format the response into JSON
    try:
        response_json = json.loads(response_text)
    except json.JSONDecodeError:
        # Handle cases where the Mistral response is not valid JSON
        raise ValueError("Mistral response is not valid JSON: " + response_text)
    
    return response_json

# Function to identify claim indices
async def identify_claims(conversation: str):
    indexed_messages = split_conversation(conversation)
    indexed_messages_str = json.dumps(indexed_messages)
    print(indexed_messages_str)
    
    claims_result = await process_message(indexed_messages_str)
    
    claim_indices = [index for index, is_claim in claims_result.items() if is_claim == 1]
    
    return claim_indices

# Function to process a CSV and output predictions to a JSON file
async def process_csv(input_csv: str, output_json: str):
    df = pd.read_csv(input_csv)
    results = []
    
    for index, row in df.iterrows():
        record_id = row['ID']
        conversation = row['Conversation']
        metadata = row["Metadata"]
        claim_indices = await identify_claims(conversation)
        
        result = {
            "record_id": record_id,
            "claim_indices": claim_indices,
            "metadata": metadata
        }
        results.append(result)
    
    with open(output_json, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"Predictions saved to {output_json}")



In [21]:
# Example usage
input_csv = "filtered_output.csv"  
output_json = "predictions.json" 
await process_csv(input_csv, output_json)

{"1": "Agent: Mike McNasby jil It looks like the volume of feedback coming from Discord has dropped to zero. The channel for issues and bugs was the primary driver, just wanted to confirm if this was intentional?", "2": "", "3": ""}
output is {"1": 1, "2": 0, "3": 0}
{"1": "User: Adding Daniela Gonz\u00e1lez, Emma's ExecOps partner. Jack Divita, can you please share some context about the meeting you'd like to set up with me and Emma Auscher and Daniela Gonz\u00e1lez can help you get it scheduled", "2": "", "3": "Agent: Hi Daniela Gonz\u00e1lez nice to be connected with you :wave:", "4": "Full context: we have the renewal of our contract with Notion coming up at the end of February and we would like to schedule some time with Emma to:", "5": "\u2022 Reflect on what we have achieved together through our partnership so far", "6": "\u2022 Share some insights and make some plans for goals for the coming year", "7": "\u2022 Walk through a proposal for the renewal contract", "8": "For each o

MistralException: Unexpected exception (ReadTimeout): The read operation timed out